In [1]:
import sys
sys.path.insert(0, '..')
%load_ext autoreload
%autoreload 2
%aimport std_func

# Hide warnings
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'pattern'

## Estimates from Cosine Similarity
We want to evaluate the feasibility of constructing optimized portfolios with the word embedding results. Our first estimate on the textual analysis is generating optimal portfolios using cosine similarity distances. We use the cosine similarity distance as correlation and sample return standard deviation to calculate the covariance estimate. We will compare the results at the end of this section to determine the feasibility.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
r_selected = pd.read_csv("data/filtered_r.csv")
# get the mean of all 
r_selected.set_index("name", inplace = True)
mu = r_selected.mean(axis = 1)
# compute the covariance matrix 
cov = r_selected.T.cov()

### Cosine Similarity Distances
We conduct cosine similarity analysis with 2-to-4 grams embeddings on the business description of each company for all top 5 SIC industry. First, we generate the words counting matrix and perform cosine similarity anlaysis to calculate the distances, which will be used as the correlation between companies in the next step for generating covarince estimate. 

In [4]:
df = pd.read_csv('../data/preprocessed.csv',
                 usecols = ['reportingDate', 'name', 'CIK',
                           'coDescription_stopwords', 'SIC', 'SIC_desc'])
df = df.set_index(df.name)

#### Words Count
For this cosine similarity analysis, we generate sequences of 2 to 4 words as one term and only select the top 600 terms by frequency.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

Vectorizer = CountVectorizer(ngram_range = (2,4), 
                             max_features = 600)

count_data = Vectorizer.fit_transform(df['coDescription_stopwords'])
wordsCount = pd.DataFrame(count_data.toarray(),columns=Vectorizer.get_feature_names())
wordsCount = wordsCount.set_index(df['name'])
wordsCount

/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,ability make,accounting standard,acquire property,act act,act amended,additional information,adequately capitalized,adverse effect,adverse effect business,adverse event,...,wa million,weighted average,well capitalized,wholly owned,wholly owned subsidiary,wide range,within day,working interest,year ended,year ended december
name,,,,,,,,,,,,,,,,,,,,,
"MONGODB, INC.",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,5,0
SALESFORCE COM INC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SPLUNK INC,0,0,0,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"OKTA, INC.",0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
VEEVA SYSTEMS INC,0,12,0,1,4,1,0,7,4,0,...,18,4,0,0,0,0,1,0,102,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"AMERICAN REALTY CAPITAL NEW YORK CITY REIT, INC.",0,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,2,2
"CYCLACEL PHARMACEUTICALS, INC.",0,0,0,0,0,1,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0
ZOETIS INC.,0,17,0,0,0,12,0,3,0,0,...,20,5,0,1,1,0,2,0,84,83


#### Cosine Similarity Computation

In [6]:
# Compute Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = pd.DataFrame(cosine_similarity(wordsCount, wordsCount))
cosine_sim = cosine_sim.set_index(df['name'])
cosine_sim.columns = df['name']
cosine_sim

name,"MONGODB, INC.",SALESFORCE COM INC,SPLUNK INC,"OKTA, INC.",VEEVA SYSTEMS INC,AUTODESK INC,"INTERNATIONAL WESTERN PETROLEUM, INC.","DAYBREAK OIL & GAS, INC.","ETERNAL SPEECH, INC.","ETERNAL SPEECH, INC.",...,OMEGA HEALTHCARE INVESTORS INC,TABLEAU SOFTWARE INC,HORIZON PHARMA PLC,MERRIMACK PHARMACEUTICALS INC,"REVEN HOUSING REIT, INC.","AMERICAN REALTY CAPITAL NEW YORK CITY REIT, INC.","CYCLACEL PHARMACEUTICALS, INC.",ZOETIS INC.,"STAG INDUSTRIAL, INC.",EQUINIX INC
name,,,,,,,,,,,,,,,,,,,,,
"MONGODB, INC.",1.000000,0.445455,0.610272,0.620961,0.500762,0.338268,0.065380,0.052345,0.000000,0.000000,...,0.050935,0.630465,0.436327,0.143385,0.066598,0.135839,0.144678,0.189609,0.178397,0.102958
SALESFORCE COM INC,0.445455,1.000000,0.635969,0.455189,0.196053,0.418546,0.043515,0.064999,0.000000,0.000000,...,0.029326,0.492079,0.300027,0.133831,0.201221,0.201230,0.145089,0.075038,0.277952,0.354856
SPLUNK INC,0.610272,0.635969,1.000000,0.665648,0.274023,0.373142,0.019112,0.073553,0.000000,0.000000,...,0.018032,0.569939,0.330028,0.116923,0.109538,0.142041,0.128467,0.136418,0.194072,0.273502
"OKTA, INC.",0.620961,0.455189,0.665648,1.000000,0.195672,0.399874,0.013240,0.093942,0.000000,0.000000,...,0.013905,0.579884,0.541775,0.163709,0.109948,0.144051,0.170361,0.111937,0.163588,0.074624
VEEVA SYSTEMS INC,0.500762,0.196053,0.274023,0.195672,1.000000,0.079927,0.074096,0.030179,0.075713,0.075713,...,0.424046,0.280852,0.153335,0.083683,0.128762,0.211695,0.060273,0.501041,0.332207,0.064207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"AMERICAN REALTY CAPITAL NEW YORK CITY REIT, INC.",0.135839,0.201230,0.142041,0.144051,0.211695,0.106627,0.027594,0.048087,0.000000,0.000000,...,0.284525,0.114080,0.075274,0.048741,0.578793,1.000000,0.039971,0.136184,0.471651,0.042298
"CYCLACEL PHARMACEUTICALS, INC.",0.144678,0.145089,0.128467,0.170361,0.060273,0.094262,0.010770,0.025407,0.000000,0.000000,...,0.015318,0.193458,0.462759,0.683597,0.047288,0.039971,1.000000,0.035694,0.080139,0.013121
ZOETIS INC.,0.189609,0.075038,0.136418,0.111937,0.501041,0.069267,0.039015,0.022235,0.065917,0.065917,...,0.159082,0.327556,0.148224,0.051060,0.163391,0.136184,0.035694,1.000000,0.207232,0.031911


### Perform Mean-Variance Analysis
We only use the Pharmaceutical Preparations industry data to generate portfolio based on Mean-Variance Analysis. We calculate the covariance estimate with cosine similarity distance as correlation and the sample standard deviation of returns. Then we use the sample return and estimated covariance to build efficient frontier.

In [7]:
!pip install PyPortfolioOpt

In [8]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions
from pypfopt import plotting

ModuleNotFoundError: No module named 'pypfopt'

In [9]:
# get the names of the companies in the pharmaceutical preparations industry
Pharm = df[df.SIC == 2834]
Pharm_list = Pharm.index

In [10]:
# get the companies name that match return data and business description data
SET = (set(Pharm_list) & set(r_selected.index))
LIST = [*SET, ]

#### Sample Mean for the Pharmaceutical Preparations Industry

In [11]:
mu_Pharm = mu[LIST]

#### Sample Covariance for the Pharmaceutical Preparations Industry

In [12]:
tmp = cov[LIST].T
cov_Pharm = tmp[LIST]

#### Cosine Similarity Distances for the Pharmaceutical Preparations Industry

In [13]:
tmp = cosine_sim[LIST].drop_duplicates().T
Pharm_cos_sim = tmp[LIST].drop_duplicates()

#### Covariance for Cosine Similarity

In [14]:
sd = pd.DataFrame(np.sqrt(np.diag(np.diagonal(cov_Pharm))))
sd = sd.set_index(cov_Pharm.index)
sd.columns = cov_Pharm.index
cos_sim_cov = pd.DataFrame((np.dot(np.dot(sd, Pharm_cos_sim),sd)))

#### Efficient Frontier - Pharmaceutical Preparations

In [15]:
ef1 = EfficientFrontier(mu_Pharm, cos_sim_cov, weight_bounds=(0, 0.2))

fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef1, ax=ax, show_assets=True)

# Find and plot the tangency portfolio
ef2 = EfficientFrontier(mu_Pharm, cos_sim_cov, weight_bounds=(0, 0.2))
# min volatility
ef2.min_volatility()
ret_tangent, std_tangent, _ = ef2.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Min Volatility")

# Format
ax.set_title("Efficient Frontier - Pharmaceutical Preparations \n Cosine Similarity Estimates")
ax.legend()
plt.tight_layout()
plt.savefig('images/Efficient_Frontier_Cos_Sim.png', dpi=200, bbox_inches='tight')
plt.show()

NameError: name 'EfficientFrontier' is not defined

##### Min Volatility Portfolio

###### Performance

In [16]:
ef2.portfolio_performance(verbose=True);

NameError: name 'ef2' is not defined

###### Weights

In [17]:
companies = []
weights = []
for company, weight in ef2.clean_weights().items():
    if weight != 0:
        companies.append(company)
        weights.append(weight)
        
dic = {'Company_Name':companies,'Weight':weights}
min_vol = pd.DataFrame(dic)
min_vol = pd.DataFrame(dic)
min_vol.to_csv("data/min_vol_cos_sim_Pharmaceutical_Preparations.csv", index = False)

NameError: name 'ef2' is not defined

In [18]:
pd.read_csv("data/min_vol_cos_sim_Pharmaceutical_Preparations.csv")

,Company_Name,Weight
0,JOHNSON & JOHNSON,0.18756
1,BIOSPECIFICS TECHNOLOGIES CORP,0.07394
2,XENCOR INC,0.02108
3,"PACIRA PHARMACEUTICALS, INC.",0.01883
4,"HERON THERAPEUTICS, INC. /DE/",0.00906
5,ZOETIS INC.,0.20000
6,PERRIGO CO PLC,0.01497
7,ARQULE INC,0.00068
8,LILLY ELI & CO,0.03562
9,BIOMARIN PHARMACEUTICAL INC,0.04572


### Results for the Other 4 Industries

In [19]:
SIC_list = [7372, 1311, 6798, 6022]
SIC_desc = ['Prepackaged Software (mass reproduction of software)', 'Crude Petroleum and Natural Gas', 
           'Real Estate Investment Trusts', 'State Commercial Banks (commercial banking)']

#### Prepackaged Software (mass reproduction of software)

In [20]:
SIC = SIC_list[0]
    
industry_name = SIC_desc[SIC_list.index(SIC)]
    
# get the names of the companies in the other industries
Companies = df[df.SIC == SIC]
Company_list = Companies.index

# get the companies name that match return data and business description data
SET = (set(Company_list) & set(r_selected.index))
LIST = [*SET, ]

mu_sample = mu[LIST]
# get the outliers
outlier = mu_sample[mu_sample>1].index
mu_sample = mu_sample.drop(outlier)
LIST = mu_sample.index

tmp = cov[LIST].T
cov_sample = tmp[LIST]

tmp = cosine_sim[LIST].T
tmp = tmp[~tmp.index.duplicated(keep="first")]
industry_cos_sim = tmp[LIST].T
industry_cos_sim = industry_cos_sim[~industry_cos_sim.index.duplicated(keep="first")]

sd = pd.DataFrame(np.sqrt(np.diag(np.diagonal(cov_sample))))
sd = sd.set_index(cov_sample.index)
sd.columns = cov_sample.index
cos_sim_cov = pd.DataFrame((np.dot(np.dot(sd, industry_cos_sim),sd)))

# perform minimum variance analysis
ef1 = EfficientFrontier(mu_sample, cos_sim_cov, weight_bounds=(0, 0.2))

fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef1, ax=ax, show_assets=True)

# Find and plot the tangency portfolio
ef2 = EfficientFrontier(mu_sample, cos_sim_cov, weight_bounds=(0, 0.2))
# min volatility
ef2.min_volatility()
ret_tangent, std_tangent, _ = ef2.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Min Volatility")

# Format
ax.set_title("Efficient Frontier - %s \n Cosine Similarity Estimates" %industry_name)
ax.legend()
plt.tight_layout()
plt.savefig('images/Efficient_Frontier_Cosine_Similarity_Estimates' + str(industry_name) + '.png', dpi=200, bbox_inches='tight')
plt.show()

NameError: name 'EfficientFrontier' is not defined

##### Min Volatility Portfolio

###### Performance

In [21]:
ef2.portfolio_performance(verbose=True);

NameError: name 'ef2' is not defined

###### Weights

In [22]:
companies = []
weights = []
for company, weight in ef2.clean_weights().items():
    if weight != 0:
        companies.append(company)
        weights.append(weight)
        
dic = {'Company_Name':companies,'Weight':weights}
min_vol = pd.DataFrame(dic)
min_vol.to_csv("data/min_vol_cos_sim_Prepackaged_Software.csv", index = False)

NameError: name 'ef2' is not defined

In [23]:
pd.read_csv("data/min_vol_cos_sim_Prepackaged_Software.csv")

,Company_Name,Weight
0,ANSYS INC,0.15390
1,"BLACK KNIGHT, INC.",0.20000
2,"ACI WORLDWIDE, INC.",0.04754
3,REALPAGE INC,0.02937
4,AWARE INC /MA/,0.02316
5,ULTIMATE SOFTWARE GROUP INC,0.10350
6,GSE SYSTEMS INC,0.04031
7,"Q2 HOLDINGS, INC.",0.06190
8,NATIONAL INSTRUMENTS CORP,0.09372
9,"INTELLICHECK, INC.",0.00938


#### Crude Petroleum and Natural Gas
When we conduct the same analysis, there is no weight shown. Efficient frontier cannot be found.

#### Real Estate Investment Trusts

In [24]:
SIC = SIC_list[2]
    
industry_name = SIC_desc[SIC_list.index(SIC)]
    
# get the names of the companies in the other industries
Companies = df[df.SIC == SIC]
Company_list = Companies.index

# get the companies name that match return data and business description data
SET = (set(Company_list) & set(r_selected.index))
LIST = [*SET, ]

mu_sample = mu[LIST]
# get the outliers
outlier = mu_sample[mu_sample>1].index
mu_sample = mu_sample.drop(outlier)
LIST = mu_sample.index

tmp = cov[LIST].T
cov_sample = tmp[LIST]

tmp = cosine_sim[LIST].T
tmp = tmp[~tmp.index.duplicated(keep="first")]
industry_cos_sim = tmp[LIST].T
industry_cos_sim = industry_cos_sim[~industry_cos_sim.index.duplicated(keep="first")]

sd = pd.DataFrame(np.sqrt(np.diag(np.diagonal(cov_sample))))
sd = sd.set_index(cov_sample.index)
sd.columns = cov_sample.index
cos_sim_cov = pd.DataFrame((np.dot(np.dot(sd, industry_cos_sim),sd)))

# perform minimum variance analysis
ef1 = EfficientFrontier(mu_sample, cos_sim_cov, weight_bounds=(0, 0.2))

fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef1, ax=ax, show_assets=True)

# Find and plot the tangency portfolio
ef2 = EfficientFrontier(mu_sample, cos_sim_cov, weight_bounds=(0, 0.2))
# min volatility
ef2.min_volatility()
ret_tangent, std_tangent, _ = ef2.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Min Volatility")

# Format
ax.set_title("Efficient Frontier - %s \n Cosine Similarity Estimates" %industry_name)
ax.legend()
plt.tight_layout()
plt.savefig('images/Efficient_Frontier_Cosine_Similarity_Estimates' + str(industry_name) + '.png', dpi=200, bbox_inches='tight')
plt.show()

NameError: name 'EfficientFrontier' is not defined

##### Min Volatility Portfolio

###### Performance

In [25]:
ef2.portfolio_performance(verbose=True);

NameError: name 'ef2' is not defined

###### Weights

In [26]:
companies = []
weights = []
for company, weight in ef2.clean_weights().items():
    if weight != 0:
        companies.append(company)
        weights.append(weight)
        
dic = {'Company_Name':companies,'Weight':weights}
min_vol = pd.DataFrame(dic)
min_vol.to_csv("data/min_vol_cos_sim_Real_Estate_Investment_Trusts.csv", index = False)

NameError: name 'ef2' is not defined

In [27]:
pd.read_csv("data/min_vol_cos_sim_Real_Estate_Investment_Trusts.csv")

,Company_Name,Weight
0,HIGHWOODS PROPERTIES INC,0.05347
1,RAYONIER INC,0.00876
2,PUBLIC STORAGE,0.06339
3,EQUINIX INC,0.07068
4,HMG COURTLAND PROPERTIES INC,0.03187
5,"GAMING & LEISURE PROPERTIES, INC.",0.06734
6,GREAT AJAX CORP.,0.13806
7,EASTGROUP PROPERTIES INC,0.01785
8,ANNALY CAPITAL MANAGEMENT INC,0.05094
9,ALEXANDERS INC,0.01487


#### State Commercial Banks (commercial banking)

In [28]:
SIC = SIC_list[3]
    
industry_name = SIC_desc[SIC_list.index(SIC)]
    
# get the names of the companies in the other industries
Companies = df[df.SIC == SIC]
Company_list = Companies.index

# get the companies name that match return data and business description data
SET = (set(Company_list) & set(r_selected.index))
LIST = [*SET, ]

mu_sample = mu[LIST]
# get the outliers
outlier = mu_sample[mu_sample>1].index
mu_sample = mu_sample.drop(outlier)
LIST = mu_sample.index

tmp = cov[LIST].T
cov_sample = tmp[LIST]

tmp = cosine_sim[LIST].T
tmp = tmp[~tmp.index.duplicated(keep="first")]
industry_cos_sim = tmp[LIST].T
industry_cos_sim = industry_cos_sim[~industry_cos_sim.index.duplicated(keep="first")]

sd = pd.DataFrame(np.sqrt(np.diag(np.diagonal(cov_sample))))
sd = sd.set_index(cov_sample.index)
sd.columns = cov_sample.index
cos_sim_cov = pd.DataFrame((np.dot(np.dot(sd, industry_cos_sim),sd)))

# perform minimum variance analysis
ef1 = EfficientFrontier(mu_sample, cos_sim_cov, weight_bounds=(0, 0.2))

fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef1, ax=ax, show_assets=True)

# Find and plot the tangency portfolio
ef2 = EfficientFrontier(mu_sample, cos_sim_cov, weight_bounds=(0, 0.2))
# min volatility
ef2.min_volatility()
ret_tangent, std_tangent, _ = ef2.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Min Volatility")

# Format
ax.set_title("Efficient Frontier - %s \n Cosine Similarity Estimates" %industry_name)
ax.legend()
plt.tight_layout()
plt.savefig('images/Efficient_Frontier_Cosine_Similarity_Estimates' + str(industry_name) + '.png', dpi=200, bbox_inches='tight')
plt.show()

NameError: name 'EfficientFrontier' is not defined

##### Min Volatility Portfolio

###### Performance

In [29]:
ef2.portfolio_performance(verbose=True);

NameError: name 'ef2' is not defined

###### Weights

In [30]:
companies = []
weights = []
for company, weight in ef2.clean_weights().items():
    if weight != 0:
        companies.append(company)
        weights.append(weight)
        
dic = {'Company_Name':companies,'Weight':weights}
min_vol = pd.DataFrame(dic)
min_vol.to_csv("data/min_vol_cos_sim_State_Commercial_Banks.csv", index = False)

NameError: name 'ef2' is not defined

In [31]:
pd.read_csv("data/min_vol_cos_sim_State_Commercial_Banks.csv")

,Company_Name,Weight
0,HOWARD BANCORP INC,0.02931
1,"CB FINANCIAL SERVICES, INC.",0.02883
2,CITIZENS & NORTHERN CORP,0.11305
3,INVESTAR HOLDING CORP,0.16789
4,EAST WEST BANCORP INC,0.08342
5,BANK OF HAWAII CORP,0.04935
6,BANNER CORP,0.20000
7,INDEPENDENT BANK CORP /MI/,0.09540
8,ENTERPRISE FINANCIAL SERVICES CORP,0.07078
9,S&T BANCORP INC,0.05201
